In [1]:
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import pandas as pd

In [2]:
Index = [10, 1, 6, 6, 10, 1, 1, 6, 10, 1, 6, 6, 6, 6, 6, 6, 10, 1, 6, 6,
         7, 7, 7, 7, 10, 10, 10, 10, 10]

attack_models = [
    ("BIM", [0.01, 0.1, 0.2, 0.3]),
    ("FGSM", [0.01, 0.1, 0.2, 0.3]),
    ("PGD", [0.01, 0.1, 0.2, 0.3]),
    ("DF", [0.01, 0.1, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.1, 0.2, 0.3]),
    ("BA", [0.01, 0.1, 0.2, 0.3]),
    # ("HopSkipJump", ["/"]),
    ("ZOO", [0, 0.01, 0.1, 0.2, 0.3])
]

base_path_template = "/home/jovyan/Cyber/UNSW/transfer_attack/x_test_adv_{attack}_eps_{eps}.npy"

file_paths = []
for attack, eps_values in attack_models:
    for eps in eps_values:
        path = base_path_template.format(attack=attack, eps=eps)
        file_paths.append(path)

file_y_pairs = []
for idx, path in enumerate(file_paths):
    file_y_pairs.append((path, Index[idx]))

eps_0_1_pairs = [(path, Index[idx]) for idx, (path, _) in enumerate(file_y_pairs) if "_eps_0.1.npy" in path]
other_eps_pairs = [(path, Index[idx]) for idx, (path, _) in enumerate(file_y_pairs) if "_eps_0.1.npy" not in path]

eps_0_1_array = np.array(eps_0_1_pairs)
other_eps_array = np.array(other_eps_pairs)

def load_data(data_array):
    X = []
    y = []
    for file_path, label in data_array:
        data = np.load(file_path)
        X.append(data)
        y.extend([int(label)] * data.shape[0])
    X = np.vstack(X)
    y = np.array(y)
    return X, y

# Load eps = 0.1 data
X, y = load_data(eps_0_1_pairs)

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y)

y = label_encoder.transform(y)
# y_UNSW = label_encoder.transform(y_UNSW)

from sklearn.model_selection import train_test_split
x_train, x_temp, y_train, y_temp = train_test_split(X, y, test_size = 0.4, random_state = 0, stratify=y)
x_test, x_val, y_test, y_val = train_test_split(x_temp, y_temp, test_size = 0.5, random_state = 0, stratify=y_temp)

In [3]:
for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '6' encode 1
defense '7' encode 2
defense '10' encode 3


In [4]:
from xgboost import XGBClassifier

print('xgb')
xgb = XGBClassifier()
xgb.fit(x_train, y_train)


xgb


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [5]:
import numpy as np
from sklearn.utils import shuffle

y_test = np.load('/home/jovyan/Cyber/UNSW/y_test.npy')

attack_models = [
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("BA", [0.01, 0.2, 0.3]),
    ("ZOO", [0, 0.01, 0.2, 0.3])
]

base_path_template = "/home/jovyan/Cyber/UNSW/transfer_attack/x_test_adv_{attack}_eps_{eps}.npy"

from sklearn.model_selection import train_test_split


for attack, eps_values in attack_models:
    X = np.load(f'/home/jovyan/Cyber/Automated_Selection/UNSW/Random_Data/x_test_adv_{attack}.npy')
    y = np.load(f'/home/jovyan/Cyber/Automated_Selection/UNSW/Random_Data/y_test_adv_{attack}.npy')
    
    x_pred = xgb.predict(X)
    unique_classes = np.unique(x_pred)
    print(unique_classes)
    for class_num in unique_classes:
        indices = np.where(x_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape)
        print(y_class.shape)
    
        x_filename = f"/home/jovyan/Cyber/Automated_Selection/UNSW/Selected_Data/x_test_adv_{attack}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Cyber/Automated_Selection/UNSW/Selected_Data/y_test_adv_{attack}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} tp {x_filename} and {y_filename}")

[0 1 2 3]
(120337, 56)
(120337,)
Save 0 tp /home/jovyan/Cyber/Automated_Selection/UNSW/Selected_Data/x_test_adv_BIM_Def1.npy and /home/jovyan/Cyber/Automated_Selection/UNSW/Selected_Data/y_test_adv_BIM_Def1.npy
(219006, 56)
(219006,)
Save 1 tp /home/jovyan/Cyber/Automated_Selection/UNSW/Selected_Data/x_test_adv_BIM_Def2.npy and /home/jovyan/Cyber/Automated_Selection/UNSW/Selected_Data/y_test_adv_BIM_Def2.npy
(2911, 56)
(2911,)
Save 2 tp /home/jovyan/Cyber/Automated_Selection/UNSW/Selected_Data/x_test_adv_BIM_Def3.npy and /home/jovyan/Cyber/Automated_Selection/UNSW/Selected_Data/y_test_adv_BIM_Def3.npy
(113630, 56)
(113630,)
Save 3 tp /home/jovyan/Cyber/Automated_Selection/UNSW/Selected_Data/x_test_adv_BIM_Def4.npy and /home/jovyan/Cyber/Automated_Selection/UNSW/Selected_Data/y_test_adv_BIM_Def4.npy
[0 1 2 3]
(114358, 56)
(114358,)
Save 0 tp /home/jovyan/Cyber/Automated_Selection/UNSW/Selected_Data/x_test_adv_FGSM_Def1.npy and /home/jovyan/Cyber/Automated_Selection/UNSW/Selected_Data/y_

In [6]:
X.shape

(569855, 56)